<a href="https://colab.research.google.com/github/heriswn/LatihanDTS/blob/master/0208_Kelompok_Invalid_Syntax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----
#  <center> Tugas DTS ITB 2019
----
### Kelompok:
  1. Ganang Iqbal R
  2. Muttaqien Rodhiya Robbi
  3. Heri Setiawan

----
#  <center> Project 1.  Yeast Classification using Random Forest Classifier
----

## Goals: Predicting the Cellular Localization Sites of Proteins

##### Attribute Information
0. Sequence Name: Accession number for the SWISS-PROT database
1. mcg: McGeoch's method for signal sequence recognition.
2. gvh: von Heijne's method for signal sequence recognition.
3. alm: Score of the ALOM membrane spanning region prediction program.
4. mit: Score of discriminant analysis of the amino acid content of the N-terminal region (20 residues long) of mitochondrial and non-mitochondrial proteins.
5. erl: Presence of "HDEL" substring (thought to act as a signal for retention in the endoplasmic reticulum lumen). Binary attribute.
6. pox: Peroxisomal targeting signal in the C-terminus.
7. vac: Score of discriminant analysis of the amino acid content of vacuolar and extracellular proteins.
8. nuc: Score of discriminant analysis of nuclear localization signals of nuclear and non-nuclear proteins.
9. Cellular Localization Sites of Proteins

In [0]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as plt
import seaborn as sns

In [0]:
data=pd.read_csv("yeast.data",header=None,delim_whitespace=True)

In [0]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


# Checking Data

In [0]:
for i in range(9):
    nr=data[data[i]=='?'].shape[0]
    if nr>0:
        print(i,':',nr)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 10 columns):
0    1484 non-null object
1    1484 non-null float64
2    1484 non-null float64
3    1484 non-null float64
4    1484 non-null float64
5    1484 non-null float64
6    1484 non-null float64
7    1484 non-null float64
8    1484 non-null float64
9    1484 non-null object
dtypes: float64(8), object(2)
memory usage: 116.0+ KB


In [0]:
data.describe()

,1,2,3,4,5,6,7,8
count,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000
mean,0.500121,0.499933,0.500034,0.261186,0.504717,0.007500,0.499885,0.276199
std,0.137299,0.123924,0.086670,0.137098,0.048351,0.075683,0.057797,0.106491
min,0.110000,0.130000,0.210000,0.000000,0.500000,0.000000,0.000000,0.000000
25%,0.410000,0.420000,0.460000,0.170000,0.500000,0.000000,0.480000,0.220000
50%,0.490000,0.490000,0.510000,0.220000,0.500000,0.000000,0.510000,0.220000
75%,0.580000,0.570000,0.550000,0.320000,0.500000,0.000000,0.530000,0.300000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.830000,0.730000,1.000000


No missing values

In [0]:
for i in range(len(data.columns)):
    if (data[i].dtypes in ['int64','float64']):
        print('\nAttribute-',i,':',data[i].dtypes)
        Q1=data[i].quantile(0.25)
        print('Q1',Q1)
        Q3=data[i].quantile(0.75)
        print('Q3',Q3)
        IQR=Q3-Q1
        print('IQR',IQR)
        min=data[i].min()
        max=data[i].max()
        min_IQR=Q1-1.5*IQR
        max_IQR=Q3+1.5*IQR
        if (min<min_IQR):
            print('Low outlier is found')
        if (max>max_IQR):
            print('High outlier is found')


Attribute- 1 : float64
Q1 0.41
Q3 0.58
IQR 0.16999999999999998
Low outlier is found
High outlier is found

Attribute- 2 : float64
Q1 0.42
Q3 0.57
IQR 0.14999999999999997
Low outlier is found
High outlier is found

Attribute- 3 : float64
Q1 0.46
Q3 0.55
IQR 0.09000000000000002
Low outlier is found
High outlier is found

Attribute- 4 : float64
Q1 0.17
Q3 0.32
IQR 0.15
High outlier is found

Attribute- 5 : float64
Q1 0.5
Q3 0.5
IQR 0.0
High outlier is found

Attribute- 6 : float64
Q1 0.0
Q3 0.0
IQR 0.0
High outlier is found

Attribute- 7 : float64
Q1 0.48
Q3 0.53
IQR 0.050000000000000044
Low outlier is found
High outlier is found

Attribute- 8 : float64
Q1 0.22
Q3 0.3
IQR 0.07999999999999999
Low outlier is found
High outlier is found


In [0]:
data.shape

(1484, 10)

In [0]:
data[9].value_counts()

CYT    463
NUC    429
MIT    244
ME3    163
ME2     51
ME1     44
EXC     35
VAC     30
POX     20
ERL      5
Name: 9, dtype: int64

In [0]:
data=data.iloc[:,1:]

In [0]:
data.shape

(1484, 9)

In [0]:
data.head()

,1,2,3,4,5,6,7,8,9
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT


In [0]:
from imblearn.over_sampling import RandomOverSampler
import collections as clc

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
X=data.iloc[:,:-1].values
Y=data.iloc[:,-1].values

ros = RandomOverSampler()
X, Y = ros.fit_sample(X, Y)

In [0]:
Y=np.reshape(Y,(4630,1))

In [0]:
data=np.concatenate([X,Y],axis=1)
data=pd.DataFrame(data)

In [0]:
data.head()

,0,1,2,3,4,5,6,7,8
0,0.58,0.61,0.47,0.13,0.5,0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0,0.48,0.22,MIT


In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [0]:
models = {
"knn": KNeighborsClassifier(n_neighbors=1),
"naive_bayes": GaussianNB(),
"logit": LogisticRegression(solver="lbfgs", multi_class="auto"),
"svm": SVC(kernel="rbf", gamma="auto"),
"decision_tree": DecisionTreeClassifier(),
"random_forest": RandomForestClassifier(n_estimators=100),
}

In [0]:
print("[INFO] loading data...")
X=data.iloc[:,:-1].values
Y=data.iloc[:,-1].values
model_names=["knn","naive_bayes","logit","svm","decision_tree","random_forest"]
targets=['CYT','NUC','MIT','ME3','ME2','ME1','EXC','VAC','POX','ERL']

(trainX, testX, trainY, testY) = train_test_split(X,Y, random_state=3, test_size=0.25)

[INFO] loading data...


In [0]:
def modfoldclass(x,y,n,mod):
    for i in range(len(mod)):
        kf=KFold(n_splits=n)
        kf.get_n_splits(x)
        model=models[mod[i]]
        j=0
        accuracy=[]
        print('Hasil dari model '+mod[i]+'\n')
        for train_index,test_index in kf.split(x):
            X_train,X_test=x[train_index],x[test_index]
            Y_train,Y_test=y[train_index],y[test_index]
            
            model.fit(X_train,Y_train)
            Y_pred=model.predict(X_test)
            
            accuracy.append(accuracy_score(Y_test,Y_pred))
            print('Subset ',j+1,' accuracy= ',accuracy[j])
            j+=1
            
        print('Average accuracy= ',np.mean(accuracy))
        print('\n')

In [0]:
result=modfoldclass(X,Y,5,model_names)

Hasil dari model knn

Subset  1  accuracy=  0.6371490280777538
Subset  2  accuracy=  0.7505399568034558
Subset  3  accuracy=  1.0
Subset  4  accuracy=  1.0
Subset  5  accuracy=  1.0
Average accuracy=  0.8775377969762419


Hasil dari model naive_bayes

Subset  1  accuracy=  0.326133909287257
Subset  2  accuracy=  0.46976241900647947
Subset  3  accuracy=  0.7213822894168467
Subset  4  accuracy=  0.023758099352051837
Subset  5  accuracy=  0.5345572354211663
Average accuracy=  0.4151187904967603


Hasil dari model logit



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Subset  1  accuracy=  0.38120950323974084


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Subset  2  accuracy=  0.6166306695464363


...

Subset  3  accuracy=  0.09719222462203024


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Subset  4  accuracy=  0.23974082073434125


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Subset  5  accuracy=  0.2159827213822894
Average accuracy=  0.3101511879049676


Hasil dari model svm

Subset  1  accuracy=  0.3174946004319654
Subset  2  accuracy=  0.6069114470842333
Subset  3  accuracy=  0.09719222462203024
Subset  4  accuracy=  0.11123110151187905
Subset  5  accuracy=  0.2451403887688985
Average accuracy=  0.27559395248380125


Hasil dari model decision_tree

Subset  1  accuracy=  0.6123110151187905
Subset  2  accuracy=  0.7278617710583153
Subset  3  accuracy=  1.0
Subset  4  accuracy=  1.0
Subset  5  accuracy=  1.0
Average accuracy=  0.8680345572354211


Hasil dari model random_forest

Subset  1  accuracy=  0.6900647948164147
Subset  2  accuracy=  0.7721382289416847
Subset  3  accuracy=  1.0
Subset  4  accuracy=  1.0
Subset  5  accuracy=  1.0
Average accuracy=  0.8924406047516198




----
#  <center> Project 2: Metro Interstate Trafic Volume Dataset
----

## Import the Data

In [0]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [0]:
#read the data
df = pd.read_csv("Metro_Interstate_Traffic_Volume.csv", sep=",", parse_dates=['date_time'])
#declare date_time format
pd.to_datetime(df['date_time'], format="%Y-%d-%m %H::%M:%S")
##change the date into day
df['day'] = df.apply(lambda row: row[7].strftime('%A'), axis=1)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,day
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,Tuesday
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,Tuesday
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,Tuesday
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,Tuesday
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,Tuesday


## Check the Data

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 10 columns):
holiday                48204 non-null object
temp                   48204 non-null float64
rain_1h                48204 non-null float64
snow_1h                48204 non-null float64
clouds_all             48204 non-null int64
weather_main           48204 non-null object
weather_description    48204 non-null object
date_time              48204 non-null datetime64[ns]
traffic_volume         48204 non-null int64
day                    48204 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 3.7+ MB


In [0]:
df.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,48204.000000,48204.000000,48204.000000,48204.000000,48204.000000
mean,281.205870,0.334264,0.000222,49.362231,3259.818355
std,13.338232,44.789133,0.008168,39.015750,1986.860670
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.160000,0.000000,0.000000,1.000000,1193.000000
50%,282.450000,0.000000,0.000000,64.000000,3380.000000
75%,291.806000,0.000000,0.000000,90.000000,4933.000000
max,310.070000,9831.300000,0.510000,100.000000,7280.000000


In [0]:
df['holiday'].value_counts()

None                         48143
Labor Day                        7
Thanksgiving Day                 6
New Years Day                    6
Martin Luther King Jr Day        6
Christmas Day                    6
Memorial Day                     5
Columbus Day                     5
Veterans Day                     5
Independence Day                 5
State Fair                       5
Washingtons Birthday             5
Name: holiday, dtype: int64

In [0]:
# create def for category to number 0/1 at holiday row
def translate_holiday(df):
    if df['holiday'] == 'None':
        return 0
    else:
        return 1
# create code holiday
df['holiday_code']=df.apply(translate_holiday,axis=1)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,day,holiday_code
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,Tuesday,0
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,Tuesday,0
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,Tuesday,0
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,Tuesday,0
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,Tuesday,0


In [0]:
#hide the columns which we have been change to another columns
df=df.drop(columns='holiday')
df=df.drop(columns='rain_1h')
df=df.drop(columns='snow_1h')
df=df.drop(columns='date_time')
df.head()

,temp,clouds_all,weather_main,weather_description,traffic_volume,day,holiday_code
0,288.28,40,Clouds,scattered clouds,5545,Tuesday,0
1,289.36,75,Clouds,broken clouds,4516,Tuesday,0
2,289.58,90,Clouds,overcast clouds,4767,Tuesday,0
3,290.13,90,Clouds,overcast clouds,5026,Tuesday,0
4,291.14,75,Clouds,broken clouds,4918,Tuesday,0


## Identify outlier

In [0]:
#identify the rest outlier
for i in df.columns:
    if (i!='holiday_code') and (df[i].dtypes in ['int64', 'float64']):
        print('\nAttribute-',i,':',df[i].dtypes)
        Q1=df[i].quantile(0.25)
        print('Q1', Q1)
        Q3=df[i].quantile(0.75)
        print('Q3', Q3)
        IQR=Q3-Q1
        print('IQR', IQR)
        min=df[i].min()
        max=df[i].max()
        min_IQR=Q1-1.5*IQR
        max_IQR=Q3+1.5*IQR
        if(min<min_IQR):
            print('Low outlier is found')
            print("min : ", min_IQR)
        if(max>max_IQR):
            print('High outlier is found')
            print("max : ", max_IQR)


Attribute- temp : float64
Q1 272.16
Q3 291.806
IQR 19.645999999999958
Low outlier is found
min :  242.6910000000001

Attribute- clouds_all : int64
Q1 1.0
Q3 90.0
IQR 89.0

Attribute- traffic_volume : int64
Q1 1193.0
Q3 4933.0
IQR 3740.0


## Drop outlier

In [0]:
#drop outlier
df_no_outlier = df[df["temp"]>242.7]
print ('[PERBANDINGAN UKURAN DATA]')
print ('shape without oulier:', df_no_outlier.shape)
print ('shape with outlier  :', df.shape)

[PERBANDINGAN UKURAN DATA]
shape without oulier: (48194, 7)
shape with outlier  : (48204, 7)


# DATA MODELLING

## Encode

In [0]:
#change the trafic_volume position
df['traffic_vol'] = df['traffic_volume']
df=df.drop(columns='traffic_volume')
#encode data dengan outlier
le = preprocessing.LabelEncoder()
for i in df.columns:
    if (df[i].dtypes=='object'):
        le.fit(df[i])
        df[i]=le.transform(df[i])
df.head()

,temp,clouds_all,weather_main,weather_description,day,holiday_code,traffic_vol
0,288.28,40,1,24,5,0,5545
1,289.36,75,1,2,5,0,4516
2,289.58,90,1,19,5,0,4767
3,290.13,90,1,19,5,0,5026
4,291.14,75,1,2,5,0,4918


In [0]:
#change the trafic_volume position
df_no_outlier['traffic_vol'] = df_no_outlier['traffic_volume']
df_no_outlier=df_no_outlier.drop(columns='traffic_volume')
#encode data dengan withoutoutlier
ge = preprocessing.LabelEncoder()
for i in df_no_outlier.columns:
    if (df_no_outlier[i].dtypes=='object'):
        ge.fit(df_no_outlier[i])
        df_no_outlier[i]=ge.transform(df_no_outlier[i])
df_no_outlier.head()

,temp,clouds_all,weather_main,weather_description,day,holiday_code,traffic_vol
0,288.28,40,1,24,5,0,5545
1,289.36,75,1,2,5,0,4516
2,289.58,90,1,19,5,0,4767
3,290.13,90,1,19,5,0,5026
4,291.14,75,1,2,5,0,4918


### Train Dataset

In [0]:
#train data with outlier
X1 = df.iloc[:, :-1].values
Y1 = df.iloc[:, -1].values

(trainX1, testX1, trainY1, testY1)= train_test_split(X1, Y1, random_state=3, test_size=0.25)

#train data without oulier
A1 = df_no_outlier.iloc[:, :-1].values
B1 = df_no_outlier.iloc[:, -1].values

(trainA1, testA1, trainB1, testB1)= train_test_split(A1, B1, random_state=3, test_size=0.25)
print("Berhasil ditrain gans...")

Berhasil ditrain gans...


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

model = LinearRegression()
model.fit(trainX1, trainY1)

print('Hasil dari model LinearRegression')
print("\nErrornya segini gan yang ada outliernya:")
predictions = model.predict(testX1)
print('MAE  =',mean_absolute_error(testY1, predictions))
print('MSE  =',mean_squared_error(testY1, predictions))
print('RMSE =',math.sqrt(mean_squared_error(testY1, predictions)))
print('--------------------------')

model.fit(trainA1, trainB1)

print("Errornya segini gan yang tidak ada outliernya:")
predictions = model.predict(testA1)
print('MAE  =',mean_absolute_error(testB1, predictions))
print('MSE  =',mean_squared_error(testB1, predictions))
print('RMSE =',math.sqrt(mean_squared_error(testB1, predictions)))

Hasil dari model LinearRegression

Errornya segini gan yang ada outliernya:
MAE  = 1706.3849124128662
MSE  = 3832602.590530008
RMSE = 1957.7033969756521
--------------------------
Errornya segini gan yang tidak ada outliernya:
MAE  = 1706.0564590459776
MSE  = 3828124.01794691
RMSE = 1956.559229348018


### K-Fold Train Dataset

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
from statistics import mean

modelsr = {"linear_r": LinearRegression()}
modnamer = ["linear_r"]

X2 = df.iloc[:, :-1].values
Y2 = df.iloc[:, -1].values
A2 = df_no_outlier.iloc[:, :-1].values
B2 = df_no_outlier.iloc[:, -1].values

def modelkfold(x, y, n, mod):
    for i in range(len(mod)):
        kf = KFold(n_splits = n)
        kf.get_n_splits(x)
        model = modelsr[mod[i]]
        
        meanAbsoluteError = []
        meanSquareError = []
        rootMeanSquareError = []
        j = 0
        print('Hasil dari model ' + mod[i] + '\n')
        for train_index, test_index in kf.split(x):
            X_train, X_test = x[train_index], x[test_index]
            Y_train, Y_test = y[train_index], y[test_index]
            
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            
            meanAbsoluteError.append(mean_absolute_error(Y_test, Y_pred))
            meanSquareError.append(mean_squared_error(Y_test, Y_pred))
            rootMeanSquareError.append(math.sqrt(mean_squared_error(Y_test, Y_pred)))
            
            print('Fold ', j+1, ' MAE  = ', meanAbsoluteError[j])
            print('Fold ', j+1, ' MSE  = ', meanSquareError[j])
            print('Fold ', j+1, ' RMSE = ', rootMeanSquareError[j])
            print('-------------------------------------')
        
            j = j + 1
    
        print("Rata-rata MAE  = ", mean(meanAbsoluteError))
        print("Rata-rata MSE  = ", mean(meanSquareError))
        print("Rata-rata RMSE = ", mean(rootMeanSquareError))
        print('\n')

In [42]:
modelkfold(X2, Y2, 5, modnamer)

Hasil dari model linear_r

Fold  1  MAE  =  1759.5171635757763
Fold  1  MSE  =  4035633.5751005705
Fold  1  RMSE =  2008.8886417869385
-------------------------------------
Fold  2  MAE  =  1693.770862868243
Fold  2  MSE  =  3796919.096092816
Fold  2  RMSE =  1948.5684735448267
-------------------------------------
Fold  3  MAE  =  1688.0568958707497
Fold  3  MSE  =  3760847.238466728
Fold  3  RMSE =  1939.2903956000835
-------------------------------------
Fold  4  MAE  =  1694.5802157253706
Fold  4  MSE  =  3791213.771139774
Fold  4  RMSE =  1947.1039446161508
-------------------------------------
Fold  5  MAE  =  1671.2971407402038
Fold  5  MSE  =  3697711.791927314
Fold  5  RMSE =  1922.9435228126993
-------------------------------------
Rata-rata MAE  =  1701.4444557560687
Rata-rata MSE  =  3816465.0945454403
Rata-rata RMSE =  1953.3589956721398




In [43]:
modelkfold(A2, B2, 5, modnamer)

Hasil dari model linear_r

Fold  1  MAE  =  1758.4078214858414
Fold  1  MSE  =  4032915.880461082
Fold  1  RMSE =  2008.2121104258588
-------------------------------------
Fold  2  MAE  =  1691.1436356719319
Fold  2  MSE  =  3779818.4239605484
Fold  2  RMSE =  1944.1755126429682
-------------------------------------
Fold  3  MAE  =  1686.879665219238
Fold  3  MSE  =  3762441.602571975
Fold  3  RMSE =  1939.7014209851925
-------------------------------------
Fold  4  MAE  =  1692.6551908813103
Fold  4  MSE  =  3788475.2452950864
Fold  4  RMSE =  1946.4005870568078
-------------------------------------
Fold  5  MAE  =  1670.920379187932
Fold  5  MSE  =  3701016.20759889
Fold  5  RMSE =  1923.802538619515
-------------------------------------
Rata-rata MAE  =  1700.0013384892507
Rata-rata MSE  =  3812933.4719775165
Rata-rata RMSE =  1952.4584339460685




------
### <center> Simpulan
------

Model LinearRegression dengan K-Fold pada data dengan menghilangkan outlier pada kolom 'temp' menghasilkan MEA, MSE, dan RMSE lebih kecil, sehingga model yang digunakan **model LinearRegression dengan K-Fold dengan data tanpa outlier**